In [1]:
import pandas as pd
import numpy as np
import os
import utm

# 1. Fact Tables 

### 1.1 ) Progreso Diario DataFrame

In [2]:
# Define the directory path for fact tables using a Windows path
path_ft = r'C:\Users\ANDRE\Desktop\Data Project (Python - Power BI)\Raw Data\Fact Tables'

# Replace backslashes with forward slashes in the path for compatibility
path_ft = path_ft.replace('\\', '/')

# Retrieves a list of files in the specified directory
ft_files = os.listdir(path_ft)

# Initializes an empty DataFrame to store concatenated data
df = pd.DataFrame()

# Loop through each file in the 'ft_files' list
for file in ft_files:
    # Reads each CSV file, specifying delimiter and encoding
    current_file = pd.read_csv(path_ft+'/'+file, sep=';', encoding='latin-1')
    
    # Unpivots the CSV data from wide to long format
    unpivot_cf = current_file.melt(
        id_vars=current_file.columns[0:4],  # Keeps the first 4 columns as identifier variables
        var_name='fecha',  # Renames the pivoted columns to 'fecha'
        value_name='valores'  # Renames the values to 'valores'
    )
    
    # Concatenates the unpivoted DataFrame to the main DataFrame 'df'
    df = pd.concat([df, unpivot_cf], axis=0)


# Data Cleaning and Transformation:

# Filtering out rows where 'valores' column is not null
df = df[df.valores.notnull()]

# Converting the 'fecha' column from object to datetime format
df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')

# Sorting the table based on specified columns
df = df.sort_values(['cod_colegio', 'meac', 'partida', 'fecha'])

# Correcting incorrect values in the 'valores' column (cumulative progress)
df['valores'].replace('fao', '0.10', inplace=True)  # Replacing 'fao' with a value lower than 0.1683 or zero
df['valores'] = df['valores'].astype(float)  # Converting the 'valores' column from object to float

# Adjusting cumulative progress values
df['valores'] = df.groupby(['cod_colegio','meac','partida']).valores.cummax()

# Calculating cumulative progress based on '%_incidencia'
df['av_acumulado'] = round(df['%_incidencia'] * df.valores, 4)

# Aggregating daily progress for each module
df = df.groupby(['cod_colegio','meac','fecha'], as_index=False).av_acumulado.sum()
df['av_acumulado'] = round(df.av_acumulado, 4)

# Calculating daily progress
df['av_diario'] = df.groupby(['cod_colegio','meac']).av_acumulado.diff().fillna(df.av_acumulado)
df['av_diario'] = round(df.av_diario, 4)

# Creating a new column with the primary keys for this table, it will be the merge of 'colegio' and 'meac'
df['colegio_meac'] = df.cod_colegio.astype(str) +'-' + df.meac.astype(str)

# Load the final Fact Table
progreso_diario = df.copy()

# Defining the directory path for cleaned data storage
path_cleaned_data = r'C:\Users\ANDRE\Desktop\Data Project (Python - Power BI)\Cleaned Data'

# Converting backslashes to forward slashes in the path for compatibility
path_cleaned_data = path_cleaned_data.replace('\\', '/')

# Saving the DataFrame 'df' to a CSV file in the cleaned data directory
df.to_csv(path_cleaned_data + '/progreso_diario.csv', index=False)

### 1.2) Rendimientos DataFrame 

In [3]:
# Based on the previous table of project performance data, we need to extract the performance of each project.

def tiempo_ejecucion_efectivo(group):
    """
    This function filters out projects that have not yet been completed.

    Args:
        group: A pandas DataFrame groupby object.

    Returns:
        A pandas DataFrame with the filtered projects.
    """

    # Create a mask to filter out projects that have not yet been completed.
    mask = group[group.av_acumulado >= 0.95].iloc[1:].index

    # Filter out the projects that have not yet been completed.
    filtered_table = group[~group.index.isin(mask)]

    # Return the filtered projects.
    return filtered_table

# Group the data by colegio and meac.
rendimientos = df.groupby(['cod_colegio','meac'], as_index=False).apply(tiempo_ejecucion_efectivo).reset_index(drop=True)

# Calculate the average daily performance of each project.
rendimientos = rendimientos.groupby(['cod_colegio','meac'], as_index=False).agg(
    av_acumulado = ('av_diario','sum'),
    rendimiento = ('av_diario','mean'),
    dias_en_ejecucion = ('av_diario','count'),
)

# Create a column that classifies each project as either 'Ejecutado' (completed) or 'En Ejecución' (in progress).
rendimientos['estado_ejecucion'] = rendimientos.av_acumulado.apply(lambda row: 'Ejecutado' if row >= 0.95 else 'En Ejecución')

# Round the rendimiento column to 4 decimal places.
rendimientos['rendimiento'] = round(rendimientos.rendimiento, 4)

# Create a column that combines the cod_colegio and meac columns into a single column.
rendimientos['colegio_meac'] = rendimientos.cod_colegio.astype(str) + '-' + rendimientos.meac.astype(str)

# Round the av_acumulado column to 4 decimal places.
rendimientos['av_acumulado'] = round(rendimientos.av_acumulado, 4)

# Save the resultados DataFrame to a CSV file.
rendimientos.to_csv(path_cleaned_data + '/rendimientos.csv', index=False)


# 2. Dimension Tables 

In [4]:
# 1. Set the path to the directory containing the dimension tables.
path_dt = r'C:\Users\ANDRE\Desktop\Data Project (Python - Power BI)\Raw Data\Dimension Tables'
path_dt = path_dt.replace('\\', '/')  # Ensure path uses forward slashes for consistency

# 2. List all files in the directory and filter out only the CSV files.
dt_files = os.listdir(path_dt)  # Get a list of all files in the directory
csv_dt_files = [file for file in dt_files if file.endswith('.csv')]  # Keep only CSV files

# 3. Create an empty dictionary to store the data from the CSV files.
dt_data = {}

# 4. Loop through each CSV file and read it into a DataFrame.
for file in csv_dt_files:
    file_name = file[:-4]  # Extract the file name without the .csv extension
    dt_data[file_name] = pd.read_csv(path_dt + '/' + file, sep=';', encoding='latin-1')  # Read the CSV file

# 5. Create global variables for each DataFrame in the dictionary.
for key, value in dt_data.items():
    globals()[key] = value  # Make the DataFrames accessible globally


## 2.1) Colegios DataFrame 

In [5]:
# Split the latitud_longitud column into two separate columns, latitud and longitud.
colegios[['latitud', 'longitud']] = colegios['latitud_longitud'].str.split(',', expand=True)


# Drop the latitud_longitud column as it is no longer needed.
colegios.drop(columns=['latitud_longitud'], inplace=True)

# Convert the latitud and longitud columns to float data type.
colegios['latitud'] = colegios['latitud'].astype(float)
colegios['longitud'] = colegios['longitud'].astype(float)

# Calculate the Easting and Northing coordinates for each colegio using UTM projection.
for index, row in colegios.iterrows():
    easting, northing, _, _ = utm.from_latlon(row.latitud, row.longitud, force_zone_number=18)
    colegios.loc[index, 'este'] = easting
    colegios.loc[index, 'norte'] = northing

    
    
# Defining execution status of each school based on the execution status of each module within it.
# If all the modules are executed, the school is executed; If even one module remains to be executed, the school continues in execution.
 
# Calculate the colegio status by grouping by cod_colegio and estado_ejecucion and counting the number of rows for each group.
colegio_status = rendimientos.groupby(['cod_colegio','estado_ejecucion'], as_index=False).estado_ejecucion.value_counts()

# Add a new column called compare_to which shows the previous estado_ejecucion for each colegio.
colegio_status['compare_to']=colegio_status.groupby('cod_colegio')['estado_ejecucion'].shift(-1).fillna(colegio_status.estado_ejecucion)

# Get the first row for each colegio, which contains the latest estado_ejecucion and compare_to values.
colegio_status = colegio_status.groupby('cod_colegio',as_index=False)[['estado_ejecucion','compare_to']].first()

# Create a new column called stado_ejecucion which is 'En Ejecución' if the current or previous estado_ejecucion is 'En Ejecución', otherwise it is 'Ejecutado'.
colegio_status['stado_ejecucion'] = colegio_status.apply(lambda row: 'En Ejecución' if (row.estado_ejecucion == 'En Ejecución') or (row.compare_to == 'En Ejecución') else 'Ejecutado', axis=1)

# Keep only the cod_colegio and stado_ejecucion columns.
colegio_status = colegio_status.loc[:,['cod_colegio','stado_ejecucion']]

# Merge the colegio_status DataFrame with the colegios DataFrame on the cod_colegio column.
colegios = colegios.merge(colegio_status, on='cod_colegio', how='left')

# Fill any missing values in the stado_ejecucion column with 'Pendiente'.
colegios['stado_ejecucion'] = colegios['stado_ejecucion'].fillna('Pendiente')

# Rename the stado_ejecucion column to estado_ejecución.
colegios.rename(columns={'stado_ejecucion':'estado_ejecución'}, inplace=True)

# Rename the estado column to estado_pronied.
colegios.rename(columns={'estado': 'estado_pronied'}, inplace=True)





# Save the colegios DataFrame to a CSV file.
colegios.to_csv(path_cleaned_data + '/colegios.csv', index=False)


## 2.2) Contratistas DataFrame 

In [6]:
# 1. Export the updated 'contratistas' DataFrame to a CSV file named 'contratistas.csv' 
contratistas.to_csv(path_cleaned_data + '/contratistas.csv', index=False)

## 2.3) MEACS DataFrame 

In [7]:
# 1. Create a new DataFrame called meac_status containing only the relevant columns from rendimientos.
meac_status = rendimientos.loc[:,['cod_colegio','meac','estado_ejecucion']]

# 2. Merge the meacs and meac_status DataFrames on the common columns 'cod_colegio' and 'meac'.
meacs = meacs.merge(meac_status, on=['cod_colegio', 'meac'], how='left')

# 3. Fill any missing values in the 'estado_ejecucion' column with 'Pendiente'.
meacs['estado_ejecucion'] = meacs.estado_ejecucion.fillna('Pendiente')

# 4. Combine the 'cod_colegio' and 'meac' columns into a single column called 'colegio_meac'.
meacs['colegio_meac'] = meacs.cod_colegio.astype(str) + '-' + meacs.meac.astype(str)

# 5. Save the meacs DataFrame to a CSV file named 'meacs.csv' without index and using 'latin-1' encoding.
meacs.to_csv(path_cleaned_data + '/meacs.csv', index=False)


## Residentes DataFrame 

In [8]:
# 1. Export the updated 'residentes' DataFrame to a CSV file named 'residente.csv' 
residentes.to_csv(path_cleaned_data +'/residentes.csv', index=False)

## Residentes Junior DataFrame 

In [9]:
# 1. Export the updated 'residentes_junior' DataFrame to a CSV file named 'residentes_junior.csv' 
residentes_junior.to_csv(path_cleaned_data+'/residentes_junior.csv', index=False)


## Plazo de Ejecución DataFrame 

In [10]:
# 1. Drop columns starting from the third column onward in the 'plazo_ejecucion' DataFrame.
plazo_ejecucion.drop(columns=plazo_ejecucion.columns[3:], inplace=True)

# 2. Convert the 'date' column to datetime format using the specified date format ('%d/%m/%Y').
plazo_ejecucion['date'] = pd.to_datetime(plazo_ejecucion['date'], format='%d/%m/%Y')

# 3. Derive additional date-related columns:
plazo_ejecucion['year'] = plazo_ejecucion['date'].dt.year
plazo_ejecucion['month_num'] = plazo_ejecucion['date'].dt.month
plazo_ejecucion['month'] = plazo_ejecucion['date'].dt.strftime('%b')
plazo_ejecucion['week'] = plazo_ejecucion['date'].dt.strftime('%V').astype(int) - 8
plazo_ejecucion['day_num'] = plazo_ejecucion['date'].dt.dayofweek + 1
plazo_ejecucion['day'] = plazo_ejecucion['date'].dt.strftime('%a')


# 4. Create cumulative_sheduled
plazo_ejecucion['programado_acumulado'] = plazo_ejecucion.programado_diario.cumsum()



# 5. Export the modified 'plazo_ejecucion' DataFrame to a CSV file named 'plazo_ejecucion.csv' in the specified directory.
plazo_ejecucion.to_csv(path_cleaned_data + '/plazo_ejecucion.csv', index=False)

# 3. Factless tables 

In [11]:

# 1. Set the path to the directory containing factless tables and replace backslashes with forward slashes for uniformity in the file path representation.
path_factless_tables = r'C:\Users\ANDRE\Desktop\Data Project (Python - Power BI)\Raw Data\Factless Fact Tables'
path_factless_tables = path_factless_tables.replace('\\', '/')

# 2. List all files in the directory and filter out only the CSV files.
factless_files = os.listdir(path_factless_tables)

# 3. Create an empty dictionary 'factless_data' to store DataFrames loaded from CSV files.
factless_data = {}

# 4. Iterate through each file in the directory:
#    - Check if the file ends with '.csv'.
#    - If it's a CSV file, read it into a Pandas DataFrame using ';' as the separator and 'latin-1' as the encoding.
#    - Store the DataFrame in the 'factless_data' dictionary with the file name (without extension) as the key.
for file in factless_files:
    if file.endswith('.csv'):
        name_file = file[:-4]
        factless_data[name_file] = pd.read_csv(path_factless_tables + '/' + file, sep=';', encoding='latin-1')

# 5. Create global variables based on the keys of the 'factless_data' dictionary, assigning them the corresponding DataFrames.
for key, value in factless_data.items():
    globals()[key] = value

# 6. Perform modifications on specific DataFrames:
#    - Concatenate 'cod_colegio' and 'meac' columns as 'colegio_meac' in 'asignacion_contratistas' DataFrame.
asignacion_contratistas['colegio_meac'] = asignacion_contratistas['cod_colegio'].astype(str) + '-' + asignacion_contratistas['meac'].astype(str)

#    - Export 'asignacion_contratistas' DataFrame to a CSV file named 'asignacion_contratistas.csv' in the specified directory.
asignacion_contratistas.to_csv(path_cleaned_data + '/asignacion_contratistas.csv', index=False)

#    - Export 'asignacion_residentes' DataFrame to a CSV file named 'asignacion_residentes.csv' in the specified directory.
asignacion_residentes.to_csv(path_cleaned_data + '/asignacion_residentes.csv', index=False)

#    - Concatenate 'cod_colegio' and 'meac' columns as 'colegio_meac' in 'asignacion_residentes_junior' DataFrame.
asignacion_residentes_junior['colegio_meac'] = asignacion_residentes_junior['cod_colegio'].astype(str) + '-' + asignacion_residentes_junior['meac'].astype(str)

#    - Export 'asignacion_residentes_junior' DataFrame to a CSV file named 'asignacion_residentes_junior.csv' in the specified directory.
asignacion_residentes_junior.to_csv(path_cleaned_data + '/asignacion_residentes_junior.csv', index=False)
